<h1 style="color:#8B66FF;">Question 1</h1>
<h2 style="color:#659FFF;">Morphing</h2>
This is the implementation of morphing algorithm used for two morph two sample faces in a way that starting from the first face,
gradually it changes to the second one in a given time interval (3s here).
<h3 style="color:#659F1F;">Input correspondences at key feature points</h3>
First I started by storing the coresponging points for the main features of faces in a file (Points.txt).
Then read the points from this file using readPoints function and the  stored the points in first and second image in pts_1, pts_2 respectively.

In [ ]:
import math
import numpy as np


def readPoints():  # read the points from file:
    with open("../images/Points.txt", "r") as file:
        n = int(file.readline())
        pts_1, pts_2 = np.ndarray((int(n / 2), 2), dtype=float), np.ndarray((int(n / 2), 2), dtype=float)
        lines = file.readlines()[0:]
        for i in range(n):
            values = lines[i].split()
            j = math.floor(i / 2)
            if i % 2 == 0: # the even lines are for the first image
                pts_1[j, 0], pts_1[j, 1] = float(values[1]), float(values[0])
            else: # the odd lines are for the second image
                pts_2[j, 0], pts_2[j, 1] = float(values[1]), float(values[0])

    file.close()
    return pts_1, pts_2, int(n / 2)

<h3 style="color:#659F1F;">Define a triangular mesh over the points</h3>
Now for triangulation, I used the inbuilt function in scipy. Notice that we should use the same mesh(triangulation) in both images.
This way, we have the triangle-to-triangle correspondence.

In [ ]:
from scipy.spatial import Delaunay

pts_1, pts_2, n = readPoints()
tri = Delaunay(pts_1)


<h3 style="color:#659F1F;">create a morphing sequence</h3>
First consider a time interval and then convert it to the [0, 1]
interval. At each step t, create an intermediate shape (by interpolation using (1 - t) * pts_1 + t * pts_2)

In [ ]:
for frame in range(1, 46):
    t = (frame - 1) / 44
    interpolation_pts = (1 - t) * pts_1 + t * pts_2
    res = np.zeros((h1, w1, 3))
    interpolate()
    out.write(res.astype(np.uint8))
    results.append(res.astype(np.uint8))



<h3 style="color:#659F1F;">Interpolate the triangles</h3>
I did this in interpolate function. First find the corresponding triangles in each image and in the average shape.
Then get the affine projection to the corresponding triangles in each image (Affine warp with three corresponding points), but before i do this, I used a rectangle around each
triangle to do this more efficiently. Then for each pixel in the triangle, find the corresponding points in each image and set value to
weighted average.


In [ ]:
def interpolate():
    tris_1 = (pts_1[tri.simplices]).astype(int)
    tris_2 = (pts_2[tri.simplices]).astype(int)
    tris_res = (interpolation_pts[tri.simplices]).astype(int)

    for i in range(tris_1.shape[0]):
        tri_1, tri_2, tri_res = tris_1[i, :], tris_2[i, :], tris_res[i, :]

        rec_1, rec_2, (x, y, w, h) = cv2.boundingRect(tri_1), cv2.boundingRect(tri_2), cv2.boundingRect(tri_res)

        tri_1[:, 0], tri_1[:, 1] = tri_1[:, 0] - rec_1[0], tri_1[:, 1] - rec_1[1]
        tri_2[:, 0], tri_2[:, 1] = tri_2[:, 0] - rec_2[0], tri_2[:, 1] - rec_2[1]
        tri_res[:, 0], tri_res[:, 1] = tri_res[:, 0] - x, tri_res[:, 1] - y

        img_rec_1 = img_1[rec_1[1]: rec_1[1] + rec_1[3], rec_1[0]: rec_1[0] + rec_1[2]]
        img_rec_2 = img_2[rec_2[1]: rec_2[1] + rec_2[3], rec_2[0]: rec_2[0] + rec_2[2]]
        mask = np.zeros((h, w, 3))
        cv2.fillConvexPoly(mask, tri_res.astype(np.int32), (1.0, 1.0, 1.0), 16, 0)

        # find the affine transform:
        size = (w, h)
        warp_mat_1 = cv2.getAffineTransform(np.float32(tri_1), np.float32(tri_res))
        warped1 = cv2.warpAffine(img_rec_1, warp_mat_1, size, None, flags=cv2.INTER_AREA, borderMode=cv2.BORDER_REFLECT)

        warp_mat_2 = cv2.getAffineTransform(np.float32(tri_2), np.float32(tri_res))
        warped2 = cv2.warpAffine(img_rec_2, warp_mat_2, size, None, flags=cv2.INTER_AREA, borderMode=cv2.BORDER_REFLECT)

        interpolated_warp = (1 - t) * warped1 + t * warped2

        res[y:y + h, x:x + w] = res[y:y + h, x:x + w] * (1 - mask) + interpolated_warp * mask

I saved the image in each iteration and used cv2.VideoWriter to make a video out of those images.